# Regression Week 5: LASSO (coordinate descent)

In this notebook, you will implement your very own LASSO solver via coordinate descent. You will:
* Write a function to normalize features
* Implement coordinate descent for LASSO
* Explore effects of L1 penalty

# Fire up sklearn and other modules

In [1]:
import math
import random
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn import model_selection
from matplotlib import pyplot as plt
%matplotlib inline

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int,
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 
              'sqft_lot':int, 'view':int}
traindt = pd.read_csv('kc_house_train_data.csv', dtype=dtype_dict)
testdt = pd.read_csv('kc_house_test_data.csv', dtype=dtype_dict)
sales = pd.read_csv('kc_house_data.csv', dtype=dtype_dict)
# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to int, before using it below
traindt['floors'] = traindt['floors'].apply(lambda x: int(float(x)))
testdt['floors'] = testdt['floors'].apply(lambda x: int(float(x)))

If we want to do any "feature engineering" like creating new features or adjusting existing ones we should do this directly using the SFrames as seen in the first notebook of Week 2. For this notebook, however, we will work with the existing features.

# Import useful functions from previous notebook

As in Week 2, we convert the SFrame into a 2D Numpy array. Copy and paste `get_num_data()` from the second notebook of Week 2.

In [3]:
def get_numpy_data(data_frame, features, output):
    data_frame['constant'] = 1
    features = ['constant'] + features
    feature_matrix = data_frame[features]
    output_array = data_frame[output]
    return (feature_matrix, output_array)

Also, copy and paste the `predict_output()` function to compute the predictions for an entire matrix of features given the matrix and the weights:

In [4]:
def predict_output(feature_matrix, weights):
    # assume feature_matrix is a numpy matrix containing the features as columns and weights is a corresponding numpy array
    # create the predictions vector by using np.dot()
    predictions = np.dot(feature_matrix, weights)      

    return(predictions)

In [5]:
a = '1'
int(a)

1

In [6]:
X = np.array([[3.,5.,8.],[4.,12.,15.]])
print (X.shape)

(2, 3)


Numpy provides a shorthand for computing 2-norms of each column:

In [7]:
norms = np.linalg.norm(X, axis=0) # gives [norm(X[:,0]), norm(X[:,1]), norm(X[:,2])]
print (norms)

[ 5. 13. 17.]


To normalize, apply element-wise division:

In [8]:
print (X / norms) # gives [X[:,0]/norm(X[:,0]), X[:,1]/norm(X[:,1]), X[:,2]/norm(X[:,2])]

[[0.6        0.38461538 0.47058824]
 [0.8        0.92307692 0.88235294]]


Using the shorthand we just covered, write a short function called `normalize_features(feature_matrix)`, which normalizes columns of a given feature matrix. The function should return a pair `(normalized_features, norms)`, where the second item contains the norms of original features. As discussed in the lectures, we will use these norms to normalize the test data in the same way as we normalized the training data. 

In [9]:
from math import sqrt
def normalize_features(x):
    x = pd.DataFrame(x)
   
    norm = np.zeros(x.shape[1])
    for c in range(x.shape[1]):
        norm[c] = sqrt(sum(x.iloc[:,c]**2))
        x.iloc[:,c] = x.iloc[:,c]/norm[c]
        
    return x, norm
    

To test the function, run the following:

In [10]:
features, norms = normalize_features(np.array([[3.,6.,9.],[4.,8.,12.]]))
print (features)
# should print
# [[ 0.6  0.6  0.6]
#  [ 0.8  0.8  0.8]]
print (norms)
# should print
# [5.  10.  15.]

     0    1    2
0  0.6  0.6  0.6
1  0.8  0.8  0.8
[ 5. 10. 15.]


In [11]:
b = 1
a = np.array([[3.,6.,9.],[4.,8.,12.]])
np.delete(a,[b], axis = 1),a

(array([[ 3.,  9.],
        [ 4., 12.]]), array([[ 3.,  6.,  9.],
        [ 4.,  8., 12.]]))

In [12]:
np.dot([1,2,3], [4,5,6])

32

# Implementing Coordinate Descent with normalized features

We seek to obtain a sparse set of weights by minimizing the LASSO cost function
```
SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|).
```
(By convention, we do not include `w[0]` in the L1 penalty term. We never want to push the intercept to zero.)

The absolute value sign makes the cost function non-differentiable, so simple gradient descent is not viable (you would need to implement a method called subgradient descent). Instead, we will use **coordinate descent**: at each iteration, we will fix all weights but weight `i` and find the value of weight `i` that minimizes the objective. That is, we look for
```
argmin_{w[i]} [ SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|) ]
```
where all weights other than `w[i]` are held to be constant. We will optimize one `w[i]` at a time, circling through the weights multiple times.  
  1. Pick a coordinate `i`
  2. Compute `w[i]` that minimizes the cost function `SUM[ (prediction - output)^2 ] + lambda*( |w[1]| + ... + |w[k]|)`
  3. Repeat Steps 1 and 2 for all coordinates, multiple times

For this notebook, we use **cyclical coordinate descent with normalized features**, where we cycle through coordinates 0 to (d-1) in order, and assume the features were normalized as discussed above. The formula for optimizing each coordinate is as follows:
```
       ┌ (ro[i] + lambda/2)     if ro[i] < -lambda/2
w[i] = ├ 0                      if -lambda/2 <= ro[i] <= lambda/2
       └ (ro[i] - lambda/2)     if ro[i] > lambda/2
```
where
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ].
```

Note that we do not regularize the weight of the constant feature (intercept) `w[0]`, so, for this weight, the update is simply:
```
w[0] = ro[i]
```

## Effect of L1 penalty

Let us consider a simple model with 2 features:

In [13]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)

Don't forget to normalize features:

In [14]:
simple_feature_matrix, norms = normalize_features(simple_feature_matrix)

We assign some random set of initial weights and inspect the values of `ro[i]`:

In [15]:
weights = np.array([1., 4., 1.])

In [16]:
simple_feature_matrix.head(2)

,constant,sqft_living,bedrooms
0,0.006802,0.003530,0.005836
1,0.006802,0.007689,0.005836


Use `predict_output()` to make predictions on this data.

In [17]:
prediction1 = predict_output(simple_feature_matrix.drop(columns = ['sqft_living'], axis = 1), np.delete(weights,1))
prediction2 = predict_output(simple_feature_matrix.drop(columns = ['bedrooms'], axis = 1), np.delete(weights,2))

Compute the values of `ro[i]` for each feature in this simple model, using the formula given above, using the formula:
```
ro[i] = SUM[ [feature_i]*(output - prediction + w[i]*[feature_i]) ]
```

*Hint: You can get a Numpy vector for feature_i using:*
```
simple_feature_matrix[:,i]
```

In [18]:
ro1 = np.dot(simple_feature_matrix.iloc[:,1], sales['price'] - prediction1 + weights[1]*simple_feature_matrix.iloc[:,1])
ro2 = np.dot(simple_feature_matrix.iloc[:,2], sales['price'] - prediction2 + weights[2]*simple_feature_matrix.iloc[:,2])
int(ro1), int(ro2), int(2*ro1), int(2*ro2)

(87939474, 80966699, 175878949, 161933399)

***QUIZ QUESTION***

Recall that, whenever `ro[i]` falls between `-l1_penalty/2` and `l1_penalty/2`, the corresponding weight `w[i]` is sent to zero. Now suppose we were to take one step of coordinate descent on either feature 1 or feature 2. What range of values of `l1_penalty` **would not** set `w[1]` zero, but **would** set `w[2]` to zero, if we were to take a step in that coordinate? 

w[2] = 0 and w[1]!=0 between non

***QUIZ QUESTION***

What range of values of `l1_penalty` would set **both** `w[1]` and `w[2]` to zero, if we were to take a step in that coordinate? 

w[2] = 0 and w[1]=0 between -161000000 to 161000000

So we can say that `ro[i]` quantifies the significance of the i-th feature: the larger `ro[i]` is, the more likely it is for the i-th feature to be retained.

## Single Coordinate Descent Step

Using the formula above, implement coordinate descent that minimizes the cost function over a single feature i. Note that the intercept (weight 0) is not regularized. The function should accept feature matrix, output, current weights, l1 penalty, and index of feature to optimize over. The function should return new weight for feature i.

In [19]:
def lasso_algo(feature_matrix, output,initial_weights, l1_norm, cycles = 1000):
    d = feature_matrix.shape[1]
    weights = np.array(initial_weights)
    featureList = feature_matrix.columns.values
    ctr = 0
    while ctr < cycles:
        ctr +=1
        for c in range(d):
            print(featureList[c],np.delete(weights,c), np.dot(feature_matrix.iloc[:,c],feature_matrix.iloc[:,c]) )
            predict = predict_output(feature_matrix.drop(columns = [featureList[c]], axis = 1), np.delete(weights,c))
            cost = np.dot(feature_matrix.iloc[:,c], predict - output) + weights[c]
            if c == 0:
                weights[c] = cost
            else:
                if cost < -1*l1_norm*0.5 :
                    weights[c] = cost + l1_norm/2

                elif cost > l1_norm*0.5 :
                    weights[c] = cost - l1_norm/2

                else:
                    weights[c] = 0
    print(weights)



In [20]:
def lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty):
    # compute prediction
    feature_matrix = pd.DataFrame(feature_matrix)
    featureList = feature_matrix.columns.values
    prediction = predict_output(feature_matrix, weights)
    # compute ro[i] = SUM[ [feature_i]*(output - prediction + weight[i]*[feature_i]) ]
    ro_i = np.dot(feature_matrix.iloc[:,i], output- prediction) + weights[i]

    if i == 0: # intercept -- do not regularize
        new_weight_i = ro_i 
    elif ro_i < -l1_penalty/2.:
        new_weight_i = ro_i + l1_penalty/2
    elif ro_i > l1_penalty/2.:
        new_weight_i = ro_i - l1_penalty/2
    else:
        new_weight_i = 0.
    
    return new_weight_i

To test the function, run the following cell:

In [21]:
# should print 0.425558846691
import math
print (lasso_coordinate_descent_step(1, np.array([[3./math.sqrt(13),1./math.sqrt(10)],[2./math.sqrt(13),3./math.sqrt(10)]]), 
                                   np.array([1., 1.]), np.array([1., 4.]), 0.1))

0.4255588466910252


## Cyclical coordinate descent 

Now that we have a function that optimizes the cost function over a single coordinate, let us implement cyclical coordinate descent where we optimize coordinates 0, 1, ..., (d-1) in order and repeat.

When do we know to stop? Each time we scan all the coordinates (features) once, we measure the change in weight for each coordinate. If no coordinate changes by more than a specified threshold, we stop.

For each iteration:
1. As you loop over features in order and perform coordinate descent, measure how much each coordinate changes.
2. After the loop, if the maximum change across all coordinates is falls below the tolerance, stop. Otherwise, go back to step 1.

Return weights

**IMPORTANT: when computing a new weight for coordinate i, make sure to incorporate the new weights for coordinates 0, 1, ..., i-1. One good way is to update your weights variable in-place. See following pseudocode for illustration.**
```
for i in range(len(weights)):
    old_weights_i = weights[i] # remember old value of weight[i], as it will be overwritten
    # the following line uses new values for weight[0], weight[1], ..., weight[i-1]
    #     and old values for weight[i], ..., weight[d-1]
    weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
    
    # use old_weights_i to compute change in coordinate
    ...
```

In [49]:
def lasso_cyclical_coordinate_descent(feature_matrix, output, initial_weights, l1_penalty, tolerance, limit):
    change = []
    success =0
    ctr = 0
    weights = initial_weights
    while success==0 :
        change = []
        ctr+=1
        #print(ctr)
        for i in range(len(weights)):
            #print(i)
            old_wt = weights[i]
            weights[i] = lasso_coordinate_descent_step(i, feature_matrix, output, weights, l1_penalty)
            change.append(abs(old_wt - weights[i]))
        print (max(change))
        if max(change) <= tolerance:
            success = 1
            print("***")
        if max(change) > limit:
            break
    print (weights)
    return weights
    

Using the following parameters, learn the weights on the sales dataset. 

In [50]:
simple_features = ['sqft_living', 'bedrooms']
my_output = 'price'
initial_weights = np.zeros(3)
l1_penalty = 1e7
tolerance = 1.0
limit = 1e25

First create a normalized version of the feature matrix, `normalized_simple_feature_matrix`.

In [51]:
(simple_feature_matrix, output) = get_numpy_data(sales, simple_features, my_output)
(normalized_simple_feature_matrix, simple_norms) = normalize_features(simple_feature_matrix) # normalize features

Then, run your implementation of LASSO coordinate descent:

In [52]:
weights = lasso_cyclical_coordinate_descent(normalized_simple_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance, limit)

79400304.63764459
9138168.376427755
8194809.518382639
6598905.081919827
5522173.230819888
4621129.84087839
3867108.1318262294
3236118.8320114315
2708086.957463354
2266213.1243878826
1896439.0751907267
1587000.4136886746
1328052.3197375722
1111356.3353527598
930018.2573927306
778268.7978372499
651280.0333459079
545011.8044225499
456083.1773606129
381664.8795189634
319389.28574656695
267275.09216532856
223664.2808007896
187169.3695841655
156629.26947971806
131072.34432779253
109685.4981489107
91788.30641868338
76811.36829749867
64278.19108917937
53790.030581101775
45013.20495937392
37668.47869067639
31522.17861736566
26378.75962406397
22074.583351306617
18472.712025832385
15458.551772344857
12936.206798683852
10825.428462021053
9059.062150921673
7580.910754904151
6343.946748185903
5308.816004425287
4442.585741579533
3717.6967622973025
3111.0866554677486
2603.4560634456575
2178.6546711325645
1823.167382284999
1525.6843353323638
1276.7410791032016
1068.4174604974687
894.0856439806521
748.1

***QUIZ QUESTIONS***
1. What is the RSS of the learned model on the normalized dataset? (Hint: use the normalized feature matrix when you make predictions.)
2. Which features had weight zero at convergence?

In [53]:
RSS = sum((output - predict_output(normalized_simple_feature_matrix, weights))**2)
RSS

1630492476715380.8

# Evaluating LASSO fit with more features

Let us split the sales dataset into training and test sets.

In [54]:
#train_data,test_data = sales.random_split(.8,seed=0)

Let us consider the following set of features.

In [55]:
all_features = ['bedrooms',
                'bathrooms',
                'sqft_living',
                'sqft_lot',
                'floors',
                'waterfront', 
                'view', 
                'condition', 
                'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 
                'yr_renovated']

First, create a normalized feature matrix from the TRAINING data with these features.  (Make you store the norms for the normalization, since we'll use them later)

In [56]:
(feature_matrix, output) = get_numpy_data(traindt, all_features, my_output)
(normalized_feature_matrix, norms) = normalize_features(feature_matrix) # normalize features

First, learn the weights with `l1_penalty=1e7`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e7`, you will need them later.

In [57]:
initial_weights = np.zeros(1+len(all_features))
tolerance = 1
l1_penalty=1e7
weights1e7 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance, limit)

71114625.71488704
5024356.370547511
5012324.915982269
5000676.42262093
4991243.174673192
4984361.344819732
4979504.930990167
4973806.5668120235
2879223.0189885423
2706150.7752847224
2532295.658294514
2368316.5870666653
2213124.3594073504
1552443.8812217042
507838.0226505175
438814.3642916903
371479.6176584661
313415.10147012025
264244.82868563384
222753.8578912504
187770.66485535353
158280.09129114076
133420.89986459538
112465.98682810739
94802.21221446618
79912.68640744686
67361.6917403005
56781.941750861704
47863.83512727171
40346.396092850715
34009.637407913804
28668.1227722764
24165.540297511965
20370.128260899335
17170.819284684956
14473.99010605365
12200.721824441105
10284.490451261401
8669.220179274678
7307.642403174192
6159.912470597774
5192.443684566766
4376.92443620041
3689.4897054173052
3110.022684838623
2621.5660897493362
2209.825927130878
1862.753202110529
1570.1913211233914
1323.5788735337555
1115.6990940608084
940.4686742685735
792.7597431913018
668.249807368964
563.2952

***QUIZ QUESTION***

What features had non-zero weight in this case?

In [58]:
featureList = pd.Series(feature_matrix.columns.values)
featureList[weights1e7!=0]

0       constant
3    sqft_living
6     waterfront
7           view
dtype: object

Next, learn the weights with `l1_penalty=1e8`, on the training data. Initialize weights to all zeros, and set the `tolerance=1`.  Call resulting weights `weights1e8`, you will need them later.

In [74]:
initial_weights = np.zeros(1+len(all_features))
l1_penalty=1e8
tolerance = 1

In [75]:
weights1e8 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance, limit)

71114625.71488704
2.9802322387695312e-08
***
[71114625.71488701        0.                0.                0.
        0.                0.                0.                0.
        0.                0.                0.                0.
        0.                0.        ]


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [60]:
featureList[weights1e8!=0]

0       constant
3    sqft_living
6     waterfront
7           view
dtype: object

In [61]:
len(all_features)

13

Finally, learn the weights with `l1_penalty=1e4`, on the training data. Initialize weights to all zeros, and set the `tolerance=5e5`.  Call resulting weights `weights1e4`, you will need them later.  (This case will take quite a bit longer to converge than the others above.)

In [72]:
initial_weights = np.zeros(1+len(all_features))
l1_penalty = 1e4
tolerance = 5e5

In [73]:
weights1e4 = lasso_cyclical_coordinate_descent(normalized_feature_matrix, output,
                                            initial_weights, l1_penalty, tolerance, 1e200)

71114625.71488704
***
[ 71114625.71488704   3956380.1252755    4963442.49245126
   5351785.21170879  -4375378.44258119  -3221806.2823531
  12795779.9345002   12003597.5806705   -8362735.52347473
   3727934.26076281   7174429.95032193   2945423.52872622
 -12246319.34070093   3043719.94683892]


***QUIZ QUESTION***

What features had non-zero weight in this case?

In [38]:
featureList[weights1e4!=0]

0          constant
1          bedrooms
2         bathrooms
3       sqft_living
4          sqft_lot
5            floors
6        waterfront
7              view
8         condition
9             grade
10       sqft_above
11    sqft_basement
12         yr_built
13     yr_renovated
dtype: object

## Rescaling learned weights

Recall that we normalized our feature matrix, before learning the weights.  To use these weights on a test set, we must normalize the test data in the same way.

Alternatively, we can rescale the learned weights to include the normalization, so we never have to worry about normalizing the test data: 

In this case, we must scale the resulting weights so that we can make predictions with *original* features:
 1. Store the norms of the original features to a vector called `norms`:
```
features, norms = normalize_features(features)
```
 2. Run Lasso on the normalized features and obtain a `weights` vector
 3. Compute the weights for the original features by performing element-wise division, i.e.
```
weights_normalized = weights / norms
```
Now, we can apply `weights_normalized` to the test data, without normalizing it!

Create a normalized version of each of the weights learned above. (`weights1e4`, `weights1e7`, `weights1e8`).

In [76]:
weights1e7n = weights1e7/norms
weights1e8n = weights1e8/norms
weights1e4n = weights1e4/norms

To check your results, if you call `normalized_weights1e7` the normalized version of `weights1e7`, then:
```
print normalized_weights1e7[3]
```
should return 161.31745624837794.

In [77]:
weights1e7n[3]

161.31745990382305

## Evaluating each of the learned models on the test data

Let's now evaluate the three models on the test data:

In [78]:
(test_feature_matrix, test_output) = get_numpy_data(testdt, all_features, 'price')

Compute the RSS of each of the three normalized weights on the (unnormalized) `test_feature_matrix`:

In [80]:
RSS1 = sum((test_output - predict_output(test_feature_matrix, weights1e7n))**2)
RSS1

275962074607804.06

In [82]:
RSS2 = sum((test_output - predict_output(test_feature_matrix, weights1e8n))**2)
RSS2

537166151497321.94

In [83]:
RSS3 = sum((test_output - predict_output(test_feature_matrix, weights1e4n))**2)
RSS3

446894731779834.56

***QUIZ QUESTION***

Which model performed best on the test data?